# 🧠 Intestinal Validator - Model Training

This notebook trains a MobileNetV3 model for photo validation.

**Steps:**
1. Download training data from Supabase
2. Train MobileNetV3 with transfer learning
3. Export to TensorFlow.js format
4. Download and deploy to web app

In [ ]:
# Install dependencies
!pip install -q tensorflow tensorflowjs supabase Pillow

In [ ]:
# Import libraries
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import tensorflowjs as tfjs
from supabase import create_client, Client

print(f"TensorFlow version: {tf.__version__}")

## 📥 Download Training Data from Supabase

In [ ]:
# Supabase credentials (use your own)
SUPABASE_URL = "your_supabase_url"
SUPABASE_KEY = "your_supabase_service_role_key"

supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)

# Create directories
os.makedirs('data/train/valid', exist_ok=True)
os.makedirs('data/train/invalid', exist_ok=True)

# Download photos
response = supabase.table('training_photos').select('*').eq('used_in_training', False).execute()
photos = response.data

print(f"Downloading {len(photos)} photos...")

for photo in photos:
    try:
        file_data = supabase.storage.from_('training-dataset').download(photo['image_url'])
        
        label = photo['label']
        filepath = f"data/train/{label}/{photo['id']}.jpg"
        
        with open(filepath, 'wb') as f:
            f.write(file_data)
        
        if len(photos) < 20 or photos.index(photo) % 10 == 0:
            print(f"Downloaded {photos.index(photo)+1}/{len(photos)}: {filepath}")
    except Exception as e:
        print(f"Error downloading {photo['image_url']}: {e}")

# Count files
valid_count = len(os.listdir('data/train/valid'))
invalid_count = len(os.listdir('data/train/invalid'))
print(f"\n✅ Dataset ready: {valid_count} valid, {invalid_count} invalid")

## 🏗️ Build Model

In [ ]:
# Create model
base_model = MobileNetV3Small(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze base model
base_model.trainable = False

# Add custom head
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

## 🎯 Train Model

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Training generator
train_generator = datagen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

# Validation generator
val_generator = datagen.flow_from_directory(
    'data/train',
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

print(f"Training samples: {train_generator.n}")
print(f"Validation samples: {val_generator.n}")

In [ ]:
# Train
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2)
    ]
)

# Print results
print(f"\n✅ Training completed!")
print(f"Final train accuracy: {history.history['accuracy'][-1]*100:.2f}%")
print(f"Final val accuracy: {history.history['val_accuracy'][-1]*100:.2f}%")

## 💾 Export Model

In [ ]:
# Save Keras model
model.save('model.h5')
print("✅ Keras model saved")

# Export to TensorFlow.js
tfjs.converters.save_keras_model(model, 'tfjs_model')
print("✅ TensorFlow.js model exported")

# Download files
from google.colab import files

print("\n📥 Downloading model files...")
files.download('tfjs_model/model.json')
files.download('tfjs_model/group1-shard1of1.bin')

print("\n✅ Done! Upload these files to your /public/model/ directory")

## 📊 Visualize Training

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()